## Reduction operation: the sum of the numbers in the range [0, value)

In [1]:
import numpy as np

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Secuencial

value = 5*10**4

X = np.random.rand(value)

# Para imprimir los pimeros valores del array

# print(X[0:12])

# Utilizando las operaciones mágicas de ipython

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("Time taken by reduction operation using a function:", tiempo)


print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")


# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")


# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum())




Time taken by reduction operation using a function: 2.62 ms ± 8.27 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
And the result of the sum of numbers in the range [0, value) is: 24995.051343753086

Time taken by reduction operation using numpy.sum(): 7.88 µs ± 8.1 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.sum(): 24995.051343752984 
 
Time taken by reduction operation using numpy.ndarray.sum(): 7.14 µs ± 0.478 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.ndarray.sum(): 24995.051343752984


 1. Con respecto a la librería multiprocessing

In [ ]:
import numpy as np 
import cupy as cp
import multiprocessing
import time
import sys

value = int(sys.argv[1])

# Se modifica la función para aceptar los parámetros de inicio y fin
def sum_multiprocessing(A, ini, fin):  # Modificada para aceptar 'ini' y 'fin'
    s = 0
    for i in range(ini, fin):
        s += A[i]
    return s

X_cpu = np.random.rand(value)

start_time = time.time()
X_gpu = cp.array(X_cpu)

result_gpu = cp.sum(X_gpu)
cp.cuda.Device().synchronize()
end_time = time.time()

result_cpu = result_gpu.item()

print(f"Tiempo total incluyendo transferencia CPU-GPU y GPU-CPU: {end_time - start_time:.4f} segundos")
print(f"Resultado de la suma en la GPU: {result_cpu}")

# Se cambia a la función starmap y se pasan los índices de inicio y fin para cada proceso
def reduc_operation_multiprocessing(n_parts, X):
    chunk_size = len(X) // n_parts
    ranges = [(i * chunk_size, (i + 1) * chunk_size) for i in range(n_parts)]
    ranges[-1] = (ranges[-1][0], len(X))  # Se ajusta el último rango
    
    # Se utiliza starmap para pasar los rangos de inicio y fin directamente
    with multiprocessing.Pool(processes=n_parts) as pool:
        results = pool.starmap(sum_multiprocessing, [(X, ini, fin) for ini, fin in ranges])
    
    return sum(results)

# Se comparan los tiempos con multiprocessing en la CPU con diferentes particiones
for n_parts in [1, 2, 4]:
    start_time = time.time()
    sum_result = reduc_operation_multiprocessing(n_parts, X_cpu)
    end_time = time.time()
    print(f"Tiempo con {n_parts} procesos (CPU): {end_time - start_time:.4f} segundos")


Time taken by reduction operation using a function: 2.6 ms ± 4.6 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
And the result of the sum of numbers in the range [0, value) is: 24890.307395611457

Time taken by reduction operation using numpy.sum(): 7.7 µs ± 0.631 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.sum(): 24890.307395611533 
 
Time taken by reduction operation using numpy.ndarray.sum(): 7.01 µs ± 48.9 ns per loop (mean ± std. dev. of 2 runs, 100,000 loops each)
Now, the result using numpy.ndarray.sum(): 24890.307395611533
Tiempo total incluyendo transferencia CPU-GPU y GPU-CPU: 1.1260 segundos
Resultado de la suma en la GPU: 24998503.108807802
Tiempo con 1 procesos (CPU): 5.8041 segundos
Tiempo con 2 procesos (CPU): 1.9136 segundos
Tiempo con 4 procesos (CPU): 1.7306 segundos

Los resultados demuestran que el uso de múltiples procesos en la CPU acelera significativamente el tiempo de ejecución en comparación con un solo proceso, logrando una mejora considerable en la eficiencia. Al aumentar el número de procesos, se aprovechan mejor los núcleos disponibles, lo que permite realizar la tarea de suma de manera mucho más rápida. En concreto, con cuatro procesos, la ejecución es varias veces más rápida que con un solo proceso, destacando la ventaja de la paralelización en entornos de CPU con múltiples núcleos.

Por otro lado, el uso de la GPU para realizar la misma operación no muestra una mejora tan significativa debido a la latencia de las transferencias de datos entre la CPU y la GPU. Aunque la GPU puede realizar cálculos más rápidos, la sobrecarga de mover los datos entre las dos memorias limita el rendimiento, haciendo que la suma en la GPU no sea tan eficiente en comparación con la paralelización de la CPU. Esto sugiere que, en tareas de reducción de datos donde los datos ya residen en la memoria de la CPU, la paralelización en la CPU podría ser más beneficiosa que el uso de la GPU, especialmente cuando se consideran las transferencias de datos.